[![preview notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/PySDM/blob/main/examples/PySDM_examples/Fisher_1991/fig_2.ipynb)
[![launch on mybinder.org](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PySDM.git/main?urlpath=lab/tree/examples/PySDM_examples/Fisher_1991/fig_2.ipynb)
[![launch on Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PySDM/blob/main/examples/PySDM_examples/Fisher_1991/fig_2.ipynb)

# based on Fig 2. in [Fisher 1991](https://doi.org/10.1034/j.1600-0889.1991.t01-4-00006.x)
Deuterium excess $d=\delta(D) - 8\delta( ^{18}O)$

In [1]:
import sys
if "google.colab" in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab("PySDM-examples")

In [2]:
from matplotlib import pyplot
import numpy as np
from open_atmos_jupyter_utils import show_plot
from scipy.integrate import solve_ivp

from PySDM import Formulae
from PySDM.physics import si
from PySDM.physics.constants import PER_MILLE, in_unit
from PySDM_examples.Jouzel_and_Merlivat_1984.thermodynamic_profiles import (
    vapour_mixing_ratio,
)

In [3]:
formulae = Formulae(
    drop_growth="Mason1971",
    diffusion_thermics="Neglect",
    latent_heat_vapourisation="Constant",
    isotope_meteoric_water_line="Dansgaard1964",
    isotope_diffusivity_ratios="Stewart1975",
    isotope_equilibrium_fractionation_factors="VanHook1968",
    isotope_kinetic_fractionation_factors="BolotEtAl2013",
)
const = formulae.constants

In [4]:
temperature = np.linspace(223.15, 263.15, 10) * si.K
saturation = 0.9
D = formulae.diffusion_thermics.D(1, 1)
K = formulae.diffusion_thermics.K(1, 1)

molar_mass_isotopes = {
    "2H": const.M_1H + const.M_2H + const.M_16O,
    "18O": 2 * const.M_1H + const.M_18O,
}
svp = formulae.saturation_vapour_pressure
Si = svp.pvs_water(temperature) / svp.pvs_ice(temperature)
r_dr_dt = formulae.drop_growth.r_dr_dt(
    RH_eq=0,
    T=temperature,
    RH=Si,
    lv=formulae.latent_heat_vapourisation.lv(temperature),
    pvs=svp.pvs_water(temperature),
    D=D,
    K=K,
)

In [5]:
alpha_l = alpha_kinetic = alpha_kinetic_eff = {}
for isotope, molar_mass in molar_mass_isotopes.items():
    alpha_l[isotope] = getattr(
        formulae.isotope_equilibrium_fractionation_factors, f"alpha_l_{isotope}"
    )(temperature)
    diffusivity_ratio = getattr(
        formulae.isotope_diffusivity_ratios, f"ratio_{isotope}"
    )(temperature)
    A_liquid_to_ice = (
        formulae.isotope_kinetic_fractionation_factors.transfer_coefficient_liq_to_ice(
            diffusion_ventilation_coefficient=D,
            condensed_water_density=1,
            pvs=svp.pvs_water(temperature),
            molar_mass=molar_mass,
            temperature=temperature,
            relative_humidity=Si,
            r_dr_dt_assuming_RHeq0=r_dr_dt,
        )
    )
    effective_saturation = (
        formulae.isotope_kinetic_fractionation_factors.effective_supersaturation(
            transfer_coefficient_liq_to_ice=A_liquid_to_ice,
            relative_humidity=Si,
        )
    )
    alpha_kinetic_eff[isotope] = (
        formulae.isotope_kinetic_fractionation_factors.alpha_kinetic(
            alpha_equilibrium=alpha_l[isotope],
            relative_humidity=effective_saturation,
            heavy_to_light_diffusivity_ratio=diffusivity_ratio,
        )
    )

In [6]:
from functools import partial
yf = partial(vapour_mixing_ratio, formulae=formulae)
alpha_s=alpha_kinetic={}
for iso in molar_mass_isotopes.keys():
    alpha_s[iso] = getattr(formulae.isotope_equilibrium_fractionation_factors, f'alpha_i_{iso}')
alpha_kinetic = formulae.isotope_kinetic_fractionation_factors.alpha_kinetic

In [7]:
def d_delta_dT(T, delta, isotope):
    y = yf(T=T)
    dT = 1 * si.K
    dy_dT = (yf(T=T+dT/2) - yf(T=T-dT/2)) / dT
    
    alpha_eq = alpha_s[isotope](T=T)
    alpha_kin = alpha_kinetic(
        alpha_equilibrium=alpha_eq,
        relative_humidity=svp.pvs_water(T) / svp.pvs_ice(T),
        heavy_to_light_diffusivity_ratio=getattr(formulae.isotope_diffusivity_ratios, f'ratio_{isotope}')
    )
    alpha = alpha_eq  * alpha_kin
    d_alpha_s_dT = (alpha(T+dt/2) - alpha(T-dT/2)) / dT

    return (
        (1 + delta)
        * (alpha * (alpha - 1) * dy_dT + y * d_alpha_s_dT)
        / alpha
        * (y + alpha * y_e)
    )

In [8]:
iso='2H'
solve_ivp(fun=partial(d_delta_dT, isotope=iso), t_span=(temperature[0], temperature[-1]), y0=[-15 * PER_MILLE])

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function truediv>) found for signature:
 
 >>> truediv(float64, type(CPUDispatcher(<function _.ratio_2H at 0x10ca36f20>)))
 
There are 8 candidate implementations:
  - Of which 6 did not match due to:
  Overload of function 'truediv': File: <numerous>: Line N/A.
    With argument(s): '(float64, type(CPUDispatcher(<function _.ratio_2H at 0x10ca36f20>)))':
   No match.
  - Of which 2 did not match due to:
  Operator Overload in function 'truediv': File: unknown: Line unknown.
    With argument(s): '(float64, type(CPUDispatcher(<function _.ratio_2H at 0x10ca36f20>)))':
   No match for registered cases:
    * (int64, int64) -> float64
    * (int64, uint64) -> float64
    * (uint64, int64) -> float64
    * (uint64, uint64) -> float64
    * (float32, float32) -> float32
    * (float64, float64) -> float64
    * (complex64, complex64) -> complex64
    * (complex128, complex